In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([transforms.ToTensor()])

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

100%|██████████| 26.4M/26.4M [00:12<00:00, 2.06MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 417kB/s]
100%|██████████| 4.42M/4.42M [00:02<00:00, 2.04MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 2.57MB/s]


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 5 * 5, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

In [4]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
for epoch in range(10):
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch:", epoch+1, "Loss:", loss.item())

Epoch: 1 Loss: 0.38860657811164856
Epoch: 2 Loss: 0.24193139374256134
Epoch: 3 Loss: 0.14616647362709045
Epoch: 4 Loss: 0.33929443359375
Epoch: 5 Loss: 0.16964368522167206
Epoch: 6 Loss: 0.057281408458948135
Epoch: 7 Loss: 0.11304927617311478
Epoch: 8 Loss: 0.06380339711904526
Epoch: 9 Loss: 0.06276583671569824
Epoch: 10 Loss: 0.1307303011417389


In [7]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Test Accuracy:", correct / total)

Test Accuracy: 0.9087
